# Structures, Brillouin zones, and cut-planning

In [ ]:
# Import packages
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import peaks as pks
import os
import pint_xarray

# Set default options
xr.set_options(cmap_sequential='Purples', keep_attrs=True)
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Example data
from peaks.core.utils.sample_data import ExampleData

## Loading structures
Handling structures and Brillouin zone plotting requires the `ase` package. This can be installed via `pip install ase` or optionally by specifying the `all` flag when installing `peaks`.

In [ ]:
# Import some core ase functions (note, if just using the basic `peaks`` functions, this is not required)
import ase
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.build import make_supercell

Passing a `.cif` file to the main `pks.load` function will load an `atoms.Atoms` object via `ase.io.read`. The full suite of `ase` utilities can then be used, e.g., to plot and visualise this structure. A couple of simple examples are shown below; see the [ase documentation](https://wiki.fysik.dtu.dk/ase/).

In [ ]:
atoms = ExampleData.structure()

In [ ]:
atoms

In [ ]:
# Interactive view of the sturcture
# view(atoms)  # Uncomment to run - opens in pop-up window

In [ ]:
# Make a supercell and plot it
scell = make_supercell(atoms, [[2, 0, 0], [0, 2, 0], [0, 0, 2]])
plot_atoms(scell, radii=0.4, rotation='90x,90y,0z')

## Brillouin zone plotting
`peaks` implements a wrapper around `ase.lattice.plot_bz`, with scaling and configurations optimised for ARPES, and some helper methods (e.g. selecting specific surface orientations, rotating the BZ to align with measured axes etc.) built in.

In [ ]:
from peaks.bz.plotting import plot_bz

The atoms object from a loaded `.cif` structure file can be passed directly, or a strucure built using standard `ase` methods. Alternatively, an `ase.lattice.BravaisLattice` object can be passed: 

In [ ]:
plot_bz(atoms, path=None)

In [ ]:
# Build a MoS2 example structure:
MoS2 = ase.build.mx2(formula='MoS2', a=3.5, size=(1, 1, 1), vacuum=None)
plot_bz(MoS2, vectors=True, rotate=15)

In [ ]:
# Make a generic lattice
lattice = ase.lattice.CUB(a=3.8)
plot_bz(lattice, path='GXM', azim=20, elev=15)

You can pass a surface orientation to plot the corresponding surface Brillouin zone, repeating as desired

In [ ]:
plot_bz(lattice, surface=(1,1,1), repeat=(2,2))

Plot on top of data by passing the relevant `matplotlib` axis

In [ ]:
FS = ExampleData.FS()
FS.metadata.set_normal_emission({'polar': '1.5 deg', 'tilt': '-0.08399999999999999 deg', 'azi': '-12.510000138999999 deg'})

In [ ]:
FSk = FS.k_convert(eV_slice=(-0.05,0.02))

In [ ]:
FSk.plot(vmax=10)
ax = plt.gca()
plot_bz(atoms, surface=(0,0,1), ax=ax)

## Brillouin zone sections
The section through multiple Brillouin zones within a given plane can be plotted:

In [ ]:
from peaks.bz.plotting import plot_bz_section

FCC = ase.lattice.FCC(a=3.8)
plot_bz_section(FCC, plane_normal=(0,0,1), plane_origin=(0,0,0), show=True)

## k-points
`sym_points` provides a quick view of the $k$ values (in $\AA^{-1}$) of high symmetry points, and optionally the angle required to get to these if measuring along the slit at a specified photon energy 

In [ ]:
from peaks.bz.utils import sym_points
sym_points(atoms, surface=(0,0,1), hv=21.2)

The methods `plot_kpar_cut` and `plot_kz_cut` can be used to append the relevant slices that will be measured to the BZ.

In [ ]:
plot_bz(atoms, surface=(0,0,1))
ax = plt.gca()
pks.plot_kpar_cut(polar=10, defl_perp=4, tilt=5, ax=ax)
pks.plot_kpar_cut(polar=5, ax=ax)
pks.plot_kpar_cut(polar=20, ax=ax, label_cut=False)

In [ ]:
plot_bz_section(FCC, plane_normal=(1,0,0), plane_origin=(0,0,0), repeat=2)
ax=plt.gca()
ax.set_xlim(-3,3)
ax.set_ylim(0,None)
ax.set_aspect('equal')
pks.plot_kz_cut(hv=200, V0=15, ax=ax)
pks.plot_kz_cut(hv=100, V0=15, ax=ax)
pks.plot_kz_cut(hv=80, V0=15, ax=ax)
pks.plot_kz_cut(hv=50, V0=15, ax=ax)